In [1]:
import pyspark

from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
        .master("local[1]") \
        .appName("test") \
        .getOrCreate()

In [5]:
spark

In [8]:
data = [("Alice", 25), ("Bob", 30), ("Charlie", 35)]
columns =["firstname","age"]

In [9]:
df = spark.createDataFrame(data =data,schema =columns)
df.show()

+---------+---+
|firstname|age|
+---------+---+
|    Alice| 25|
|      Bob| 30|
|  Charlie| 35|
+---------+---+



In [10]:
df.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- age: long (nullable = true)



In [11]:
df.filter("age>=26").show()

+---------+---+
|firstname|age|
+---------+---+
|      Bob| 30|
|  Charlie| 35|
+---------+---+



In [12]:
df.select("firstname","age").show()

+---------+---+
|firstname|age|
+---------+---+
|    Alice| 25|
|      Bob| 30|
|  Charlie| 35|
+---------+---+



In [94]:
from pyspark.sql.functions import mean as mean_, std as std_

df = spark.read.csv("avocado.csv",header=True)
df2 =spark.read.csv("avocado.csv", header= True)
df.show()
df

+---+----------+------------+------------+-------+---------+------+----------+----------+----------+-----------+------------+----+------+
|_c0|      Date|AveragePrice|Total Volume|   4046|     4225|  4770|Total Bags|Small Bags|Large Bags|XLarge Bags|        type|year|region|
+---+----------+------------+------------+-------+---------+------+----------+----------+----------+-----------+------------+----+------+
|  0|2015-12-27|        1.33|    64236.62|1036.74| 54454.85| 48.16|   8696.87|   8603.62|     93.25|        0.0|conventional|2015|Albany|
|  1|2015-12-20|        1.35|    54876.98| 674.28| 44638.81| 58.33|   9505.56|   9408.07|     97.49|        0.0|conventional|2015|Albany|
|  2|2015-12-13|        0.93|   118220.22|  794.7|109149.67| 130.5|   8145.35|   8042.21|    103.14|        0.0|conventional|2015|Albany|
|  3|2015-12-06|        1.08|    78992.15| 1132.0| 71976.41| 72.58|   5811.16|    5677.4|    133.76|        0.0|conventional|2015|Albany|
|  4|2015-11-29|        1.28|     

DataFrame[_c0: string, Date: string, AveragePrice: string, Total Volume: string, 4046: string, 4225: string, 4770: string, Total Bags: string, Small Bags: string, Large Bags: string, XLarge Bags: string, type: string, year: string, region: string]

In [95]:
from pyspark.sql.functions import mean as mean_, std as std_ , col

df_stats = df.select(
    mean_(col('AveragePrice')).alias('mean'),
    std_(col('AveragePrice')).alias('std')
).collect()

avg = df_stats[0]['mean']
std = df_stats[0]['std']
print(avg)
print(std)

df.describe().show()

1.4059784097758825
0.4026765554955525
+-------+------------------+----------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------------+----------------+
|summary|               _c0|      Date|      AveragePrice|      Total Volume|              4046|              4225|              4770|        Total Bags|        Small Bags|        Large Bags|       XLarge Bags|        type|              year|          region|
+-------+------------------+----------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------------+----------------+
|  count|             18249|     18249|             18249|             18249|             18249|             18249|             18249|             18249|             18249|          

In [41]:
df.groupBy('4046').count().orderBy('count', ascending=False).show(5)

+----+-----+
|4046|count|
+----+-----+
| 0.0|  242|
| 3.0|   10|
| 1.0|    8|
| 4.0|    8|
|1.24|    8|
+----+-----+
only showing top 5 rows



In [52]:
from pyspark.sql.functions import min as minimum_, median as median_ , max as maximum_, col
import pyspark.sql.functions as F
df_stats = df.select(
    minimum_(col('AveragePrice')).alias('min'),
    median_(col('AveragePrice')).alias('median'),
    maximum_(col('AveragePrice')).alias('max')
).collect()

min = df_stats[0]['min']
median = df_stats[0]['median']
max = df_stats[0]['max'] 
print(min)
print(median)
print(max)

df1 = df.groupby('type').agg(F.expr('percentile(AveragePrice, array(0.25))')[0].alias('%25'),
                             F.expr('percentile(AveragePrice, array(0.75))')[0].alias('%75'))

df1.show()
# df.join(df1, on='', how='left').show()

0.44
1.37
3.25
+------------+----+----+
|        type| %25| %75|
+------------+----+----+
|     organic|1.42|1.87|
|conventional|0.98|1.32|
+------------+----+----+



In [98]:
# df2.show()

df2.printSchema()
df2= df2.selectExpr("AveragePrice as avgprice")
df2.show()
df2.printSchema()
df2

root
 |-- _c0: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- AveragePrice: string (nullable = true)
 |-- Total Volume: string (nullable = true)
 |-- 4046: string (nullable = true)
 |-- 4225: string (nullable = true)
 |-- 4770: string (nullable = true)
 |-- Total Bags: string (nullable = true)
 |-- Small Bags: string (nullable = true)
 |-- Large Bags: string (nullable = true)
 |-- XLarge Bags: string (nullable = true)
 |-- type: string (nullable = true)
 |-- year: string (nullable = true)
 |-- region: string (nullable = true)

+--------+
|avgprice|
+--------+
|    1.33|
|    1.35|
|    0.93|
|    1.08|
|    1.28|
|    1.26|
|    0.99|
|    0.98|
|    1.02|
|    1.07|
|    1.12|
|    1.28|
|    1.31|
|    0.99|
|    1.33|
|    1.28|
|    1.11|
|    1.07|
|    1.34|
|    1.33|
+--------+
only showing top 20 rows

root
 |-- avgprice: string (nullable = true)



DataFrame[avgprice: string]

In [71]:
wordCountsDF = (df.select("region")
                .groupBy('region').count())
wordCountsDF.show(df.count(), False)
print((wordCountsDF).count())

counts=df.map(lambda x:[(c,1)for c in x])
for row in counts.collect():
    print .parallelize(row).reduceByKey(lambda x,y:x+y).collect()


wordsLengthsDF =data.select(length('region').alias('lengths'),"region")
wordsLengthsDF.show()

+-------------------+-----+
|region             |count|
+-------------------+-----+
|PhoenixTucson      |338  |
|GrandRapids        |338  |
|SouthCarolina      |338  |
|TotalUS            |338  |
|WestTexNewMexico   |335  |
|Louisville         |338  |
|Philadelphia       |338  |
|Sacramento         |338  |
|DallasFtWorth      |338  |
|Indianapolis       |338  |
|LasVegas           |338  |
|Nashville          |338  |
|GreatLakes         |338  |
|Detroit            |338  |
|Albany             |338  |
|Portland           |338  |
|CincinnatiDayton   |338  |
|SanDiego           |338  |
|Boise              |338  |
|HarrisburgScranton |338  |
|StLouis            |338  |
|NewOrleansMobile   |338  |
|Columbus           |338  |
|Pittsburgh         |338  |
|MiamiFtLauderdale  |338  |
|SouthCentral       |338  |
|Chicago            |338  |
|BuffaloRochester   |338  |
|Tampa              |338  |
|Southeast          |338  |
|Plains             |338  |
|Atlanta            |338  |
|BaltimoreWashington

In [106]:

df = spark.createDataFrame([ ("A", 1, None), ("B", None, "123" ), ("B", 3, "456"), ("D", None, None), ], ["Name", "Value", "id"]) 

df1 = df.na.drop(subset='Value')  
df1.show()

df2=df.na.drop(subset=["Value","id"])
df2.show()

+----+-----+----+
|Name|Value|  id|
+----+-----+----+
|   A|    1|NULL|
|   B|    3| 456|
+----+-----+----+

+----+-----+---+
|Name|Value| id|
+----+-----+---+
|   B|    3|456|
+----+-----+---+

